In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [53]:
base_url = "https://ncrdb.usga.org/courseTeeInfo?CourseID="
# all_data = []
empty_count = 0
current_id = 4

In [54]:
## Test with CGC
id_cgc = 1899
cgc_url = base_url + str(id_cgc)
response = requests.get(cgc_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [55]:
table = soup.find('table',id='gvTee')

In [56]:
header_row = table.find('tr', class_='tableRows')

In [57]:
headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

In [58]:
headers

['Tee Name',
 'Gender',
 'Par',
 'Course Rating™',
 'Bogey Rating™',
 'Slope Rating®',
 'RatingF9',
 'RatingB9',
 'Front (9)',
 'Back (9)',
 'Bogey Rating (F9)',
 'Bogey Rating (B9)',
 'Slope (F9)',
 'Slope (B9)',
 'TeeID',
 'Length',
 '',
 'CH']

In [59]:
len(headers)

18

In [22]:
metadata_table = soup.find('table', id='gvCourseTees')

In [23]:
# Extract the values from the table's row
course_name = metadata_table.find_all('td')[0].get_text(strip=True).split(" - ")[0]
city = metadata_table.find_all('td')[1].get_text(strip=True)
state = metadata_table.find_all('td')[2].get_text(strip=True)

In [24]:
print("Course Name:", course_name)
print("City:", city)
print("State:", state)

Course Name: Colorado Golf Club
City: Parker
State: CO


In [25]:
# Find all rows excluding the header row
rows = table.find_all('tr')[1:]  # Skip the header row

In [26]:
# Extract data from each row
data = []
for row in rows:
    cells = row.find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    data.append(row_data)

In [27]:
# Print extracted data
for row in data:
    print(row)

['Championship', 'M', '72', '76.5', '104.8', '152', '38.4', '38.1', '38.4 / 149', '38.1 / 155', '52.2', '52.6', '149', '149', '225891', '7571', '', '']
['Champ/Black', 'M', '72', '75.6', '103.7', '151', '37.8', '37.8', '37.8 / 147', '37.8 / 155', '51.5', '52.2', '147', '147', '165159', '7409', '', '']
['Black', 'M', '72', '75.1', '103.0', '150', '37.7', '37.4', '37.7 / 147', '37.4 / 153', '51.4', '51.6', '147', '147', '225892', '7300', '', '']
['Black/Paintbrush', 'M', '72', '74.0', '101.3', '147', '37.1', '36.9', '37.1 / 145', '36.9 / 149', '50.6', '50.7', '145', '145', '165160', '7095', '', '']
['Paintbrush', 'M', '72', '73.1', '99.3', '141', '36.6', '36.5', '36.6 / 139', '36.5 / 143', '49.5', '49.8', '139', '139', '646913', '6925', '', '']
['Paintbrush/White', 'M', '72', '70.8', '96.2', '137', '35.5', '35.3', '35.5 / 136', '35.3 / 138', '48.1', '48.1', '136', '136', '646903', '6503', '', '']
['White', 'M', '72', '69.9', '94.0', '130', '35.0', '34.9', '35.0 / 135', '34.9 / 125', '47.

In [14]:
# Create DataFrame
df = pd.DataFrame(data, columns=headers)
df = df.drop(columns='CH')

# Add the extracted values as new columns to the beginning of the DataFrame
df.insert(0, 'Course Name', course_name)
df.insert(1, 'City', city)
df.insert(2, 'State', state)

df

,Course Name,City,State,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,Slope Rating®,RatingF9,RatingB9,Front (9),Back (9),Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,
0,Colorado Golf Club,Parker,CO,Championship,M,72,76.5,104.8,152,38.4,38.1,38.4 / 149,38.1 / 155,52.2,52.6,149,149,225891,7571,
1,Colorado Golf Club,Parker,CO,Champ/Black,M,72,75.6,103.7,151,37.8,37.8,37.8 / 147,37.8 / 155,51.5,52.2,147,147,165159,7409,
2,Colorado Golf Club,Parker,CO,Black,M,72,75.1,103.0,150,37.7,37.4,37.7 / 147,37.4 / 153,51.4,51.6,147,147,225892,7300,
3,Colorado Golf Club,Parker,CO,Black/Paintbrush,M,72,74.0,101.3,147,37.1,36.9,37.1 / 145,36.9 / 149,50.6,50.7,145,145,165160,7095,
4,Colorado Golf Club,Parker,CO,Paintbrush,M,72,73.1,99.3,141,36.6,36.5,36.6 / 139,36.5 / 143,49.5,49.8,139,139,646913,6925,
5,Colorado Golf Club,Parker,CO,Paintbrush/White,M,72,70.8,96.2,137,35.5,35.3,35.5 / 136,35.3 / 138,48.1,48.1,136,136,646903,6503,
6,Colorado Golf Club,Parker,CO,White,M,72,69.9,94.0,130,35.0,34.9,35.0 / 135,34.9 / 125,47.5,46.5,135,135,225894,6228,
7,Colorado Golf Club,Parker,CO,White/Wild,M,72,66.6,89.7,125,33.7,32.9,33.7 / 127,32.9 / 122,45.5,44.2,127,127,717049,5567,
8,Colorado Golf Club,Parker,CO,Wildflower,M,72,64.8,86.9,119,32.9,31.9,32.9 / 123,31.9 / 115,44.3,42.6,123,123,225963,5187,
9,Colorado Golf Club,Parker,CO,White,F,72,75.7,111.8,153,38.5,37.2,38.5 / 154,37.2 / 152,56.7,55.1,154,154,646922,6184,


In [31]:
## Test with CGC
id_empty = 2
empty_url = base_url + str(id_empty)
response = requests.get(empty_url)
error_soup = BeautifulSoup(response.content, 'html.parser')

In [32]:
error_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Search course ratings and slope" name="Description"/>
<meta content="United States Golf Association, USGA, Golf, NCRDB, National Course Rating Database, Course Rating Lookup, Online Course Ratings" name="keywords"/>
<title>National Course Rating Database</title>
<link href="/lib/bootstrap/dist/css/bootstrap.min.css" rel="stylesheet"/>
<script src="/lib/jquery/dist/jquery.min.js"></script>
<script src="https://www.usga.org/content/usga/home-page.jsonp.js"></script>
<link href="/lib/datatables/datatables.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/usga_skinned_apps_styles.css" rel="Stylesheet" type="text/css"/>
<link href="/css/usga_skinned_apps_media_screens.css" rel="Stylesheet" type="text/css"/>
<

In [33]:
# Initialize the flag variable
error_flag = 0

In [34]:
# Check for the specific error message
error_message = soup.find('h2', class_='text-danger errorHandler')
if error_message and "An error occurred. If this problem persists, contact USGA Handicap Department." in error_message.get_text():
    error_flag = 1

In [35]:
# Output the flag variable
print(f"Error Flag: {error_flag}")

Error Flag: 1


In [95]:
## Test with 9 hole
id_9 = 566
url_9 = base_url + str(id_9)
response = requests.get(url_9)
soup_9hole = BeautifulSoup(response.content, 'html.parser')

In [96]:
metadata_table = soup_9hole.find('table', id='gvCourseTees')

In [97]:
# Extract info
course_name = metadata_table.find_all('td')[0].get_text(strip=True).split(" - ")[0]
city = metadata_table.find_all('td')[1].get_text(strip=True)
state = metadata_table.find_all('td')[2].get_text(strip=True)

In [98]:
## Course data
table = soup_9hole.find('table',id='gvTee')

In [99]:
# Header info
header_row = table.find('tr', class_='tableRows')
headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

In [100]:
headers

['Tee Name',
 'Gender',
 'Par',
 'Course Rating™',
 'Bogey Rating™',
 'Slope Rating®',
 'RatingF9',
 'RatingB9',
 'Front (9)',
 'Back (9)',
 'Bogey Rating (F9)',
 'Bogey Rating (B9)',
 'Slope (F9)',
 'Slope (B9)',
 'TeeID',
 'Length']

In [101]:
len(headers)

16

In [102]:
# Course info
rows = table.find_all('tr')[1:]  # Skip the header row

In [89]:
# Extract data from each row
data = []
for row in rows:
    cells = row.find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    data.append(row_data)

In [90]:
data[0]

['Blue',
 'M',
 '34',
 '33.7',
 '44.7',
 '118',
 '33.7',
 '',
 '33.7 / 118',
 '/',
 '44.7',
 '',
 '118',
 '118',
 '742317',
 '2873',
 '',
 '']

In [91]:
len(data[0])

18

In [117]:
max_row_data

[['Blue',
  'M',
  '34',
  '33.7',
  '44.7',
  '118',
  '33.7',
  '',
  '33.7 / 118',
  '/',
  '44.7',
  '',
  '118',
  '118',
  '742317',
  '2873',
  '',
  ''],
 ['Red',
  'M',
  '34',
  '32.4',
  '42.8',
  '112',
  '32.4',
  '',
  '32.4 / 112',
  '/',
  '42.8',
  '',
  '112',
  '112',
  '742321',
  '2570',
  '',
  ''],
 ['Blue',
  'F',
  '34',
  '36.4',
  '50.4',
  '119',
  '36.4',
  '',
  '36.4 / 119',
  '/',
  '50.4',
  '',
  '119',
  '119',
  '742322',
  '2873',
  '',
  ''],
 ['Red',
  'F',
  '34',
  '34.7',
  '47.9',
  '112',
  '34.7',
  '',
  '34.7 / 112',
  '/',
  '47.9',
  '',
  '112',
  '112',
  '742319',
  '2570',
  '',
  '']]

In [108]:
max_row_data = []
max_rows = 16  # Set the maximum number of rows to collect
row_count = 0  # Initialize a counter for the number of rows

for row in rows:
    if row_count >= max_rows:
        break  # Stop the loop if we've reached the maximum number of rows

    cells = row.find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    max_row_data.append(row_data)
    row_count += 1  # Increment the counter for each row processed

In [115]:
len(data[0])

18

In [116]:
data[0]

['Blue',
 'M',
 '34',
 '33.7',
 '44.7',
 '118',
 '33.7',
 '',
 '33.7 / 118',
 '/',
 '44.7',
 '',
 '118',
 '118',
 '742317',
 '2873',
 '',
 '']

In [118]:
# Remove the last two values from each sublist
cleaned_data = [sublist[:-2] for sublist in max_row_data]

In [119]:
len(cleaned_data[0])

16

In [122]:
## Consolidate all into df
df = pd.DataFrame(cleaned_data, columns=headers)
# df = df.drop(columns='CH')
df

,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,Slope Rating®,RatingF9,RatingB9,Front (9),Back (9),Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length
0,Blue,M,34,33.7,44.7,118,33.7,,33.7 / 118,/,44.7,,118,118,742317,2873
1,Red,M,34,32.4,42.8,112,32.4,,32.4 / 112,/,42.8,,112,112,742321,2570
2,Blue,F,34,36.4,50.4,119,36.4,,36.4 / 119,/,50.4,,119,119,742322,2873
3,Red,F,34,34.7,47.9,112,34.7,,34.7 / 112,/,47.9,,112,112,742319,2570


In [123]:
# Add the extracted values as new columns to the beginning of the DataFrame
df.insert(0, 'Course Name', course_name)
df.insert(1, 'City', city)
df.insert(2, 'State', state)

# Add ts
df['usga_course_id'] = id_9
df['timestamp'] = pd.Timestamp.now()

In [124]:
df

,Course Name,City,State,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,Slope Rating®,RatingF9,...,Front (9),Back (9),Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,usga_course_id,timestamp
0,Chosen Valley Golf Club,Chatfield,MN,Blue,M,34,33.7,44.7,118,33.7,...,33.7 / 118,/,44.7,,118,118,742317,2873,566,2024-08-08 18:33:29.342630
1,Chosen Valley Golf Club,Chatfield,MN,Red,M,34,32.4,42.8,112,32.4,...,32.4 / 112,/,42.8,,112,112,742321,2570,566,2024-08-08 18:33:29.342630
2,Chosen Valley Golf Club,Chatfield,MN,Blue,F,34,36.4,50.4,119,36.4,...,36.4 / 119,/,50.4,,119,119,742322,2873,566,2024-08-08 18:33:29.342630
3,Chosen Valley Golf Club,Chatfield,MN,Red,F,34,34.7,47.9,112,34.7,...,34.7 / 112,/,47.9,,112,112,742319,2570,566,2024-08-08 18:33:29.342630


In [125]:
df.columns

Index(['Course Name', 'City', 'State', 'Tee Name', 'Gender', 'Par',
       'Course Rating™', 'Bogey Rating™', 'Slope Rating®', 'RatingF9',
       'RatingB9', 'Front (9)', 'Back (9)', 'Bogey Rating (F9)',
       'Bogey Rating (B9)', 'Slope (F9)', 'Slope (B9)', 'TeeID', 'Length',
       'usga_course_id', 'timestamp'],
      dtype='object')

In [69]:
main_cols = ['Course Name', 'City', 'State', 'Tee Name', 'Gender', 'Par',
       'Course Rating™', 'Bogey Rating™', 'Slope Rating®', 'RatingF9',
       'RatingB9', 'Front (9)', 'Back (9)', 'Bogey Rating (F9)',
       'Bogey Rating (B9)', 'Slope (F9)', 'Slope (B9)', 'TeeID', 'Length',
       '', 'usga_course_id', 'timestamp']

In [70]:
len(main_cols)

22